In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import os
import numpy as np
import requests
import nltk

/home/ofir/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ofir/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ofir/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ofir/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ofir/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
with open('data/sentences/rt-polarity-utf8.neg') as f:
    neg = f.read().splitlines()
with open('data/sentences/rt-polarity-utf8.pos') as f:
    pos = f.read().splitlines()

In [3]:
len(neg), len(pos)

(5349, 5346)

In [4]:
data = pos + neg
target = [1 for _ in range(len(pos))] + [-1 for _ in range(len(neg))]

In [5]:
neg_train_size = int(len(neg) * 0.8)
pos_train_size = int(len(pos) * 0.8)

In [6]:
neg_train, neg_test = neg[:neg_train_size], neg[neg_train_size:]
pos_train, pos_test = pos[:pos_train_size], pos[pos_train_size:]

In [7]:
train_data = neg_train + pos_train
test_data = neg_test + pos_test

In [8]:
len(train_data) + len(test_data) == len(neg) + len(pos)

True

In [9]:
neg_target_train = [-1 for i in range(len(neg_train))]
neg_target_test = [-1 for i in range(len(neg_test))]
pos_target_train = [1 for i in range(len(pos_train))]
pos_target_test = [1 for i in range(len(pos_test))]

In [10]:
target_train = neg_target_train + pos_target_train
target_test = neg_target_test + pos_target_test

In [11]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(train_data, target_train)
predicted_svm = text_clf_svm.predict(test_data)
np.mean(predicted_svm == target_test)

/home/ofir/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.735981308411215

In [12]:
# Training Support Vector Machines - SVM and calculating its performance
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(data, target)

/home/ofir/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [13]:
pos_documents_path = "data/documents/pos"
neg_documents_path = "data/documents/neg"

In [14]:
pos_documents = []
neg_documents = []

for file_name in os.listdir(pos_documents_path):
    with open(os.path.join(pos_documents_path, file_name)) as f:
        pos_documents.append(f.read().splitlines())
        
for file_name in os.listdir(neg_documents_path):
    with open(os.path.join(neg_documents_path, file_name)) as f:
        neg_documents.append(f.read().splitlines())

In [15]:
len(pos_documents), len(neg_documents)

(1000, 1000)

In [16]:
# def add_pos_to_sentence(sentence):
#     response = requests.post("http://text-processing.com/api/tag/", data={"text": sentence, "output": "iob"})
#     if response.status_code != 200:
#         return None
#     response_list = response.json().get("text").split("\n")
#     return " ".join(["_".join(var.split(" ")[0:2]) for var in response_list])

In [17]:
def add_pos_to_sentence(sentence):
    tokenized = nltk.word_tokenize(sentence.replace("_", ""))
    return " ".join(["{}_{}".format(token[0], token[1]) for token in nltk.pos_tag(tokenized)])

In [18]:
def predict_and_add_pos_documents(documents):
    output = []
    for i, doc in enumerate(documents):
        results = text_clf_svm.predict(doc)
        for sentence, label in zip(doc, results):
            posed_sentence = add_pos_to_sentence(sentence)
            output.append("{}\t{}".format(label, posed_sentence))
        output.append("")
    return output

In [19]:
pos_output = predict_and_add_pos_documents(pos_documents[:1])
neg_output = predict_and_add_pos_documents(neg_documents[:1])

In [21]:
thefile = open('../data/pos-1.txt', 'w')
for sentence in pos_output:
    thefile.write("%s\n" % sentence)

In [22]:
thefile = open('../data/neg-1.txt', 'w')
for sentence in neg_output:
    thefile.write("%s\n" % sentence)

In [151]:
import osqp

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

ImportError: numpy.core.multiarray failed to import